In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Parkinson

In [ ]:
import os, glob
import pandas as pd
from collections import defaultdict

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [ ]:
# Rawdata 저장 경로
RAWDATAPATH = os.path.join(os.getcwd(), 'Rawdata')
RAW_CONTROL = os.path.join(RAWDATAPATH , 'Controls')  # '/content/drive/MyDrive/Parkinson/Rawdata/Controls'
RAW_PD = os.path.join(RAWDATAPATH, 'PD')              # '/content/drive/MyDrive/Parkinson/Rawdata/PD'


# 추출 후 저장할 경로
DATASETPATH = os.path.join(os.getcwd(), 'dataset')           
DATA_CONTROL = os.path.join(DATASETPATH , 'Controls')    # '/content/drive/MyDrive/Parkinson/dataset/Controls'
DATA_PD = os.path.join(DATASETPATH , 'PD')               # '/content/drive/MyDrive/Parkinson/dataset/Controls'


# CSV INFO(FW,BW 개수 및 경로) 저장 경로
SAVEPATH = os.path.join(DATASETPATH, 'patients.csv')

## 39개 관측 센서 위치
- 머리 -> 몸통 -> 다리 -> 발
- 앞면 왼,오 -> 뒷면 왼,오 순서로!

In [ ]:
MARKERS = ['LFHD', 'RFHD', 'LBHD', 'RBHD', 'C7', 'LSHO', 'RSHO', 'CLAV', 'RBAK',
           'LUPA', 'RUPA', 'STRN', 'T10', 'LELB', 'RELB', 'LFRM', 'RFRM',
           'LWRA', 'RWRA', 'LWRB', 'RWRB', 'LFIN', 'RFIN', 'LASI', 'RASI',
           'LPSI', 'RPSI', 'LTHI', 'RTHI', 'LKNE', 'RKNE', 'LTIB', 'RTIB',
           'LANK', 'RANK', 'LTOE', 'RTOE', 'LHEE', 'RHEE']
len(MARKERS)

## 데이터 추출
- Subframe 열 제거
- 두번째 행의 측정 단위를 보면 369개의 측정값 중 1/3은 거리(mm), 1/3은 속도(mm/s), 나머지 1/3은 가속도(mm/s^2)임을 알 수 있다. \
따라서, 속도와 가속도 데이터를 제외하고 추출한다.
- CentreOfMass_XYZ, CentreOfMasFloor_XYZ 제거
- 컬럼 순서 재배열 (머리 - 상체 - 하체 - 발)

- 총 컬럼 수: Frame + 마커 39개 * 3 (X,Y,Z) = 118개
- 저장 파일명: PREP_이니셜_FW/BW1-3.csv \
  (Trial 번호 1,2,3으로 다시 넘버링 함)

In [ ]:
def prep_data_from_raw(target_cate:str, target_file:str):
    """
    target_cate: 'Controls' or 'PD'
    target_file: '이니셜_FW1.csv', '이니셜_FW_01.csv', 'KangYK_FW01.csv'
    
    >> Save as './dataset/{소속}/PREP_{이니셜}_{FW/BW}{1~3}.csv' after preprocessing
    """
    print(f"\n======= Current Preprocessing File: {target_file} =======")

    # ============  데이터 추출  ===============
    # 첫 두 행(Trajectories 정보) 제외하고 읽기
    # 일단 Frame을 인덱스로 설정
    target_path = RAW_PD if target_cate == 'PD' else RAW_CONTROL
    df = pd.read_csv(os.path.join(target_path, target_file), skiprows=2, index_col=0)
    
    # Sub Frame 제거
    df.drop(columns=df.columns[df.iloc[0]=='Sub Frame'], inplace=True)  

    # 단위가 mm인 데이터 추출
    df = df[df.columns[df.iloc[1]=='mm']]  # 39*3=117개 넘으면 Centre 포함
    print(f"[Before] Number of Markers: {df.shape[1] // 3}")  # 마커 개수 확인

    # 컬럼명 변경: '이니셜:마커' or 'Unnamed: n' -> '마커명_X', '마커명_Y', '마커명_Z'
    col_names = []
    for i, col in enumerate(df.columns):
        if i % 3 == 0:
            placement = col.split(':')[1]
        col_names.append(placement + '_' + df.iloc[0,i])  # LFHD_X, LFHD_Y, LFHD_Z, ...
    df.columns = col_names
    
    # XYZ, 단위(mm) 행 제거
    df = df.iloc[2:]

    # CentreOfMass, CentreOfMasFloor, TURN_MARKER 제거
    df = df.loc[:, [col for col in df.columns if col.split('_')[0] in MARKERS]]
    assert df.shape[1] == 117
    # print(f"[After] Number of Markers: {len(df.columns) // 3} (Goal: 39)")

    # 마커 순서 재배열
    df = df.loc[:, [m + '_' + axis for m in MARKERS for axis in ['X','Y','Z']]]

    # Frame을 인덱스에서 컬럼으로 변경
    df['Frame'] = df.index
    df = df[[df.columns[-1]] + df.columns[:-1].to_list()] # Frame 컬럼을 맨 앞으로
    df.reset_index(drop=True, inplace=True)               # 인덱스 리셋 (0부터 시작)
    assert df.shape[1] == 118
    # print(f"[FINAL] Number of columns: {df.shape[1]} (Goal: 118)")


    # =============  Prep 후 csv로 저장  =============
    # 저장할 때 파일명 포맷 통일: 'PREP_이니셜_FW/BW1~3.csv'
    splitted = target_file[:-4].split('_')
    splitted.insert(0, 'PREP')
    
    if len(splitted) == 4:    # PREP_이니셜_FW_1 인 경우 (언더바 하나 더 있음)
        splitted[2] = splitted[2] + str(int(splitted[-1]))
        splitted = splitted[:3]
    assert len(splitted) == 3

    if len(splitted[-1]) > 3: # PREP_이니셜_FW01 인 경우 (숫자에 0 포함)
        splitted[-1] = splitted[-1][:2] + str(int(splitted[-1][2:]))

    # Trial Re-numbering: 124, 234, 134 등 제각각이므로 1,2,3으로 통일
    save_path = DATA_PD if target_cate == 'PD' else DATA_CONTROL
    raw_files = glob.glob(os.path.join(target_path, f"{splitted[1]}_{splitted[-1][:2]}*.csv"))
    prep_files = glob.glob(os.path.join(save_path, f"PREP_{splitted[1]}_{splitted[-1][:2]}*.csv"))
    if len(prep_files) < len(raw_files):
        splitted[-1] = splitted[-1][:2] + str(len(prep_files) + 1)
        df.to_csv(os.path.join(save_path, '_'.join(splitted) + '.csv'), encoding='utf-8', index=False)
    else:
        print(f"\n** 3 prep files of {splitted[1]}_{splitted[2][:2]} already exist!! **")

    return

In [ ]:
def generate_prep_data(rawdata_path):
    category = rawdata_path.split('/')[-1]
    for filename in sorted(os.listdir(rawdata_path)):
        # Exception
        if category == 'PD' and filename == 'KMS_FW3.csv': continue  # 원본 자체에 마커가 38개.. RFHD 누락!
        
        name, trial = filename.split('_')[:2]
        save_path = DATA_PD if category == 'PD' else DATA_CONTROL
        # if len(glob.glob(os.path.join(save_path, f"PREP_{name}_{trial[:2]}*.csv"))) == 3: 
        #     continue  # 이미 Prep한 파일이 3개 있으면 패스한다.
        
        prep_data_from_raw(category, filename)

- Preprocessing 수행 (Controls)

In [ ]:
# Prep 수행 전, 저장할 폴더의 존재여부 및 비어있는지 확인!!
# trial numbering 할 때 중복여부를 체크하지 않으므로 무조건 비어있는 상태에서 처음부터 해야함...
if not os.path.exists(DATASETPATH):
    os.mkdir(DATASETPATH)
if not os.path.exists(DATA_CONTROL):
    os.mkdir(DATA_CONTROL)

assert len(os.listdir(DATA_CONTROL)) == 0
generate_prep_data(RAW_CONTROL)

In [ ]:
print(f"[Controls] Raw: {len(os.listdir(RAW_CONTROL))} -> Prep: {len(os.listdir(DATA_CONTROL))}")
assert len(os.listdir(RAW_CONTROL)) == len(os.listdir(DATA_CONTROL))

In [ ]:
# control_names = set(f.split('_')[:2][0] for f in sorted(os.listdir(RAW_CONTROL)))
# len(control_names)

- Preprocessing 수행 (PD)

In [ ]:
# Prep 수행 전, 저장할 폴더의 존재여부 및 비어있는지 확인!!
if not os.path.exists(DATASETPATH):
    os.mkdir(DATASETPATH)
if not os.path.exists(DATA_PD):
    os.mkdir(DATA_PD)

assert len(os.listdir(DATA_PD)) == 0
generate_prep_data(RAW_PD)

In [ ]:
print(f"\n[PD] Raw: {len(os.listdir(RAW_PD))} -> Prep: {len(os.listdir(DATA_PD))}")
assert len(os.listdir(RAW_PD)) - 1 == len(os.listdir(DATA_PD))

## 환자 정보 취합

- 환자 이름: 영문 이니셜
- 소속: PD, Controls
- FW, BW 개수
- FW, BW 파일 위치: `./dataset/{소속}` (Prep 후)

In [ ]:
def get_csv_info(target_cate: str) -> pd.DataFrame:
    
    """
    target_cate: 'Controls' or 'PD'
    """

    # 실험 참여자별 FW, BW 파일 개수 세기 & 파일 경로 저장
    cntFW = defaultdict(int)
    cntBW = defaultdict(int)
    pathFW = defaultdict(list)
    pathBW = defaultdict(list)

    target_path = DATA_CONTROL if target_cate == 'Controls' else DATA_PD

    for filename in sorted(os.listdir(target_path)):
        _, name, trial = filename[:-4].split('_')  # [PREP, 이니셜, trial]
        
        if trial[:2] == 'FW':
            cntFW[name] += 1
            pathFW[name].append('/'.join(['.', DATASETPATH.split('/')[-1], target_cate, filename]))
        else:  #  trial[:2] == 'BW'
            cntBW[name] += 1
            pathBW[name].append('/'.join(['.', DATASETPATH.split('/')[-1], target_cate, filename]))
    
    # FW만 있는 환자는 BW 개수를 0으로 입력
    for k in cntFW.keys():
        if k not in cntBW.keys():
            cntBW[k] = 0
            pathBW[k] = []
    
    # 딕셔너리 정렬: 키 값(이니셜) 기준
    cntFW = dict(sorted(cntFW.items()))
    cntBW = dict(sorted(cntBW.items()))
    pathFW = dict(sorted(pathFW.items()))
    pathBW = dict(sorted(pathBW.items()))

    df = pd.DataFrame()
    df['Patient'] = list(cntFW.keys())
    df['Category'] = target_cate
    df['cntFW'] = list(cntFW.values())
    df['cntBW'] = list(cntBW.values())
    df['pathFW'] = pathFW.values()
    df['pathBW'] = pathBW.values()
    
    return df

In [ ]:
def generate_csv_info_table(save_path):
    """
    Create and save patients FW & BW count information table
    save path = ./dataset/patients.csv
    """

    df_PD = get_csv_info('PD')
    df_Controls = get_csv_info('Controls')
    df_info = pd.concat([df_PD, df_Controls], ignore_index=True)
    df_info.to_csv(save_path, encoding='utf-8', index=False)
    # return df_info

In [ ]:
generate_csv_info_table(SAVEPATH)

## 수진쌤 코드 터미널 실행

In [ ]:
%cd /content/drive/MyDrive/NIMS/Parkinson/code/utils

In [ ]:
!bash python preprocessor.py --debug=true